In [ ]:
from langchain_core.callbacks import BaseCallbackHandler
from langchain.agents import AgentExecutor, create_react_agent
from langchain_core.tools import tool
from common.utils import get_llm  # Import the get_llm function

class DetailedCallbackHandler(BaseCallbackHandler):
    def on_tool_start(self, serialized, inputs, **kwargs):
        print(f"🛠️ Tool Started: {serialized.get('name')}")
        print(f"Tool Inputs: {inputs}")

    def on_tool_end(self, output, **kwargs):
        print(f"🏁 Tool Completed. Output: {output}")

    def on_chain_start(self, serialized, inputs, **kwargs):
        print(f"🔗 Chain Started: {serialized.get('name')}")

    def on_chain_end(self, outputs, **kwargs):
        print(f"✅ Chain Completed. Outputs: {outputs}")

    def on_agent_action(self, action, **kwargs):
        print(f"🤖 Agent Action: {action.tool}")

@tool
def calculator(expression: str) -> float:
    """Evaluates a mathematical expression."""
    return eval(expression)

@tool
def greeting(name: str) -> str:
    """Returns a greeting for a given name."""
    return f"Hello, {name}!"


: 

In [ ]:

# Get LLM instance using the utility function
llm, token_counter = get_llm(model_name="gpt-3.5-turbo", temperature=0)
tools = [calculator, greeting]

agent = create_react_agent(llm, tools)
agent_executor = AgentExecutor(agent=agent, tools=tools)

callback_handler = DetailedCallbackHandler()

result = agent_executor.invoke(
    {"input": "What is 5 + 3? Then greet Alice."},
    {"callbacks": [callback_handler, token_counter]}  # Added token_counter to track token usage
)

print("\nFinal Result:", result)
print(f"\nToken Usage - Input: {token_counter.input_tokens}, Output: {token_counter.output_tokens}")